This is where I will be testing out Sean's sequencing+ model.

Notes from lookng at tjstuff+:

tjstuff+ 1.0
* machine learning model XGboosted tree which aims to predict how effective a pitch is at limiting runs by using the physical characteristics of the pitch.
* Features: speed, spin, break, etc
* Target: xRV (average run value) broken out by count and outcome (called strike, swinging strike, hit into play, foul, ball). He took average because RV for an outcome can vary significantly depending on number of outs and base state. He broke it out by count because count greatly impacts xRV.

tjstuff+ 2.0
- feature changes: extension to log_extension, fastball differentials to primary pitch differentials
- target still xRV.
- better hyper parameter tuning
- this model takes all the physical characteristics of a pitch and then tries to predict the xRV to determine how good the outcome of the pitch will be. I don't like how this doesn't fully control for count because the proportion of pitches thrown at certain points in the count will affect a pitch's perceived effectiveness. If we include count as an input, it knows there are only a few numbers it can hit for xRV, so it will be able to better predict the outcome/event given the pitch's characteristics

run expectancy is just an average the MLB (or tom tango) calculates based on number of outs, count, and runner positions. this is basically a giant matrix (see RE288). run value is the change in run expectancy. Statcast's delta_run_exp is just about how the outcome of a pitch moves from one part of the matrix to another (from one game state to another). RE falls short because it doesn't take into account batter characteristics or pitcher characteristics or previous pitches. I want to make a model to LEARN RE. A terrible batter at the plate should not have the same RE as someone with Aaron Judge's metrics. Throwing a ball that sets up a killer tunnel should influence RE differently than a ball on a wild pitch.

Stated goal:
- full model will ideally predict how many runs will be scored in a half-inning (RE) given a certain game state.
- Full model will use encoder-decoder architecture where encoder gets batter-specific information, and decoder spits out RE for different pitches.
- Specifically, the input tokens for the decoder should be pitch type, characteristics/location of pitch, the outcome of the previous pitch (ball, strike, swinging strike, foul ball), and the out/bases/count game state, and should NOT include the outcome of the current pitch (because the model is also estimating the outcome of the current pitch to determine RE).
- The targets for the decoder are the number of runs scored in the rest of the half-inning (learning RE). I don't want to use precalculated RE if I take into account batter info or pitch sequencing. But if I'm not including batter info or sequencing, I could alternatively train on the run value of the at-bat, which might be better. Training at the at-bat level might make most sense because pitchers aren’t penalized for something that hasn’t happened yet. However, half-inning level is ideal because the model would be actually learning RE. I might need to boost the training data by mixing the data around (meaning mix up at bats in an inning, not mix up pitches in an at-bat) because that seems really hard to learn.
- It might make sense to do a softmax classification head (with CE loss) if the model is predicting runs because there's only so many runs that get scored in an inning (it would only have to choose between a max of 30 classes). It would get RE by just using the expected value formula.
- If I want to do a first iteration without transformer, I will not consider outcome of previous pitch. The non-transformer model will have the input be pitch type, characteristics/location of pitch, and the out/bases/count game state. this is just a stuff+ model